In [378]:
import pandas as pd
import numpy as np
import scipy
import time
import random
import sys
# Import T. Kipf's GCN implementation
# https://github.com/tkipf/keras-gcn
sys.path.append('./keras-gcn/')
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import to_categorical
import keras.backend as K
import tensorflow as tf
from kegra.layers.graph import GraphConvolution
from kegra.utils import *
%matplotlib inline

In [379]:
X = pd.read_hdf('history_small.hdf', key='hist') 
A = pd.read_pickle('adjacency_small.pkl') 
A = A[sorted(A.columns)]

In [380]:
X.shape, A.shape

((878, 290), (878, 878))

### One student and all the graph

In [126]:
student = 142954.0
X_k_ = X[student]

In [127]:
X_k_[X_k_==1] = 10 #valid
X_k_[X_k_==0] = 1 #not valid
X_k_[X_k_==-100] = 0 #missing
np.unique(X_k_)
X = pd.get_dummies(X_k_)[[1,10]]

In [128]:
y = X_k_.ravel(order='F')

In [129]:
A = scipy.sparse.csr_matrix(A.values)
X = np.asmatrix(X)

In [130]:
ind, _ = np.where(X!=0)
ind

array([327, 328, 330, 333, 338, 340, 347, 349, 352, 355, 358, 364, 371,
       372, 380, 381, 385, 387, 436, 442])

In [131]:
#for training no missing value
#x_k_ = X_k_[X_k_!=0]
#X = pd.get_dummies(x_k_)
#A = A[ind][:,ind]
#y = y[ind]

In [132]:
print(X.shape, A.shape, y.shape)
print(type(X), type(A), type(y))

(878, 2) (878, 878) (878,)
<class 'numpy.matrixlib.defmatrix.matrix'> <class 'scipy.sparse.csr.csr_matrix'> <class 'numpy.ndarray'>


In [133]:
#A = A[ind][:,ind]

In [134]:
y_train = y.reshape(878,1)
y_val = y.reshape(878,1)
y_test = y.reshape(878,1)
idx_train = range(878)
idx_test = range(878)
idx_val = range(878)

In [135]:
# Normalize X ??

In [136]:
# Define parameters
FILTER = 'localpool'  # 'chebyshev'
MAX_DEGREE = 2  # maximum polynomial degree
SYM_NORM = True  # symmetric (True) vs. left-only (False) normalization
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience

In [137]:
A_ = preprocess_adj(A, SYM_NORM)
support = 1
graph = [X, A_]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

In [138]:
X_in = Input(shape=(X.shape[1],))
H = Dropout(0.5)(X_in)
H = GraphConvolution(16, support, activation='relu', W_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(1, support, activation='softmax')([H]+G)

In [139]:
def loss_(y_true, y_pred):
    ix = tf.where(tf.not_equal(y_true, 0))
    true = tf.gather(y_true, ix)
    pred = tf.gather(y_pred, ix)
    print(true)
    print(pred)
    return K.binary_crossentropy(true, pred)

In [149]:
def custom_loss(y_true, y_pred):
    ix = tf.where(tf.not_equal(y_true, 0))
    y_true = tf.gather(y_true, ix)
    y_true = K.concatenate([y_true, y_true], axis=0)

    y_pred = tf.gather(y_pred, ix)
    y_pred = K.concatenate([y_pred, y_pred], axis=0)

    return K.binary_crossentropy(y_true, y_pred)

In [150]:
# Compile model
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss=loss_, optimizer=Adam(lr=0.01)) #adapt crossentropy

Tensor("loss_9/graph_convolution_14_loss/Gather:0", shape=(?, 2, ?), dtype=float32)
Tensor("loss_9/graph_convolution_14_loss/Gather_1:0", shape=(?, 2, 1), dtype=float32)


In [151]:
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, #sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=1)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])
    
    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

# Testing
#test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
#print("Test set results:",
#      "loss= {:.4f}".format(test_loss[0]),
#      "accuracy= {:.4f}".format(test_acc[0]))

Epoch 1/1


/Users/sharonedayan/tensorflow/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


InvalidArgumentError: Incompatible shapes: [20] vs. [878]
	 [[Node: training_5/Adam/gradients/loss_9/graph_convolution_14_loss/mul_grad/BroadcastGradientArgs = BroadcastGradientArgs[T=DT_INT32, _class=["loc:@loss_9/graph_convolution_14_loss/mul"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](training_5/Adam/gradients/loss_9/graph_convolution_14_loss/mul_grad/Shape, training_5/Adam/gradients/loss_9/graph_convolution_14_loss/mul_grad/Shape_1)]]

Caused by op 'training_5/Adam/gradients/loss_9/graph_convolution_14_loss/mul_grad/BroadcastGradientArgs', defined at:
  File "//anaconda/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "//anaconda/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "//anaconda/lib/python3.5/site-packages/traitlets/config/application.py", line 589, in launch_inst

(index 0 : -100 (NaN), index 1 : 0 (not valid), index 2 : 1 (valid))  
index 0 : 0 (not valid), index 1 : 1 (valid)

In [85]:
(preds == 1).all()

True

#### TESTS

In [ ]:
student = 142954.0
X_k_ = X[student]
X_k, ex_k = X_k_[X_k_!=-100], X_k_[X_k_!=-100].index

In [ ]:
n_samples = 15
full = 20

X_train = X_k.values.reshape(full,1)
y_train = X_k[:full].values.reshape(full,1)

A_train = A[A.index.isin(idx_full)][sorted(idx_full)]
A_train = scipy.sparse.csr_matrix(A_train.values)

train_mask = sample_mask(range(15), A_train.shape[0]).reshape(20,1)

In [ ]:
A_train.shape, X_train.shape, y_train.shape, train_mask.shape

In [ ]:
# Normalize X_train
X_train = X_train/X_train.sum()
X_train = X_train.reshape((len(X_train), 1))

A_ = preprocess_adj(A_train, True)
support = 1
graph = [X_train, A_]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]
X_in = Input(shape=(X_train.shape[1],))

# Define model architecture
H = Dropout(0.5)(X_in)
H = GraphConvolution(16, support, activation='relu', W_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y_train.shape[1], support, activation='softmax')([H]+G)

# Compile model
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

In [ ]:
h = model.fit(graph, y_train, #sample_weight=train_mask, 
          batch_size=A_train.shape[0], epochs=1, shuffle=False, verbose=0)

In [ ]:
student = 142954.0 # np.random.choice(X.columns)
X_k_ = X[student]
X_k, ex_k = X_k_[X_k_!=-100], X_k_[X_k_!=-100].index
c = 15
assert c + 1< len(X_k)

In [ ]:
def one_hot(x):
    if x == 0:
        return [1,0]
    elif x == 1:
        return [0,1]

In [ ]:
n_samples = 15
idx_train = random.sample(list(ex_k), n_samples)
idx_test = random.sample([k for k in ex_k if k not in idx_train], 1)
idx_full = np.hstack((idx_train,idx_test))
X_train = X_k.loc[idx_full].values
#X_train = to_categorical(X_train)
#y_train = X_k.loc[idx_test]
#y_train = np.array(y_train).reshape((len(y_train), 1))
y_train = np.zeros((X_train.shape[0],2))
y_train[n_samples] = one_hot(X_train[n_samples])
A_train = A[A.index.isin(idx_full)][sorted(idx_full)]
A_train = scipy.sparse.csr_matrix(A_train.values)

In [ ]:
A_train.shape, X_train.shape, y_train.shape

In [ ]:
train_mask = np.array(list(map(bool, np.hstack((np.ones(n_samples), [0])))))

In [ ]:
# Normalize X_train
X_train = X_train/X_train.sum()
X_train = X_train.reshape((len(X_train), 1))

In [ ]:
A_ = preprocess_adj(A_train, True)
support = 1
graph = [X_train, A_]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

In [ ]:
X_in = Input(shape=(X_train.shape[1],))

# Define model architecture
H = Dropout(0.5)(X_in)
H = GraphConvolution(16, support, activation='relu', W_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y_train.shape[1], support, activation='softmax')([H]+G)

# Compile model
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

In [ ]:
h = model.fit(graph, y_train, sample_weight=train_mask,
          batch_size=A_train.shape[0], epochs=1, shuffle=False, verbose=0)

In [ ]:
preds = model.predict(graph, batch_size=A_train.shape[0])

In [ ]:
preds